# Stock Data Analysis

### Import Required Packages

In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display


### API KEY VALIDITY CHECK

In [25]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [26]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

### BUILDING URL AND RESPONSE STRUCTIRE

In [75]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"
    
    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_three_year_ago = datetime.datetime.now() - datetime.timedelta(days=1095)
        params = {"frequency": "1wk", "period1": datetime_three_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.title.string if soup.title else "No title found"

        for irrelevant in soup.body(['script', 'style', 'image', 'input']):
            irrelevant.decompose()

        table_data = soup.find('table')

        return title, table_data



### BUILDNIG SYSTEM AND USER PROMPTS

In [76]:
def build_stock_analysis_prompt(stock_symbol, table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over three years timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."

    return [
        {'role':'system', "content":sys_prompt},
        {"role":"user", "content":user_prompt}
    ]

### AGGREGATING EVERYTHING WITH OPENAI

In [83]:
def analyse_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, table_data = stock_data_page.get_stock_data()
    openai = OpenAI()
    response = openai.chat.completions.create(
        model= "gpt-4o-mini",
        messages= build_stock_analysis_prompt(stock_symbol, table_data)
    )

    return response.choices[0].message.content

In [84]:
def display_analysis(stock_symbol):
    display(Markdown(analyse_stock_trends(stock_symbol)))

### FINAL RESPONSE

In [85]:
display_analysis('GOOG')

# GOOG Stock Analysis (Jan 2022 - Feb 2025)

### Summary of the Data
The provided data contains stock information for Google (GOOG) over the period from January 2022 to February 2025. The key metrics include the opening price, highest price, lowest price, closing price, adjusted closing price, and trading volume for each week.

### Price Trends
1. **General Price Movement**
   - The stock started at around **$89.31** in late December 2022 and saw substantial growth over the period.
   - By February 21, 2025, the closing price was **$181.58**, showing strong appreciation.

2. **Major Peaks and Troughs**
   - **Highest Price:** The highest recorded price reached **$208.70** on February 3, 2025.
   - **Lowest Price:** The lowest price in the given timeframe was **$86.70** observed on October 31, 2022.

3. **Volatility**
   - There were significant fluctuations between the highest and lowest stock prices each week, notably in early 2023, indicating high volatility.
   - The stock experienced declines followed by rebounds, particularly throughout 2023 before stabilizing at a higher price point by 2024.

4. **Dividend Payments**
   - **Dividends:** GOOG made several small dividend payments averaging **$0.20**, which can contribute to overall returns alongside capital gains.

### Volume Insight
- The trading volume varied significantly, with some weeks showing high trading activity (over **200 million** shares) during noteworthy price moves, while other weeks had significantly lower volume indicating less activity.

### Calculation of Total Returns
To calculate the total return over the period, we will use the following formula:

\[
\text{Total Return (\%)} = \left( \frac{\text{Final Price} - \text{Initial Price}}{\text{Initial Price}} \right) \times 100
\]

- **Initial Price (Jan 1, 2022):** **$89.31** 
- **Final Price (Feb 21, 2025):** **$181.58**

### Calculation:
\[
\text{Total Return (\%)} = \left( \frac{181.58 - 89.31}{89.31} \right) \times 100 = \left( \frac{92.27}{89.31} \right) \times 100 \approx 103.3\%
\]

### Conclusion
- **Total Expected Return:** Approximately **103.3%** over the reviewed period.
- The stock has more than doubled in value, suggesting strong performance for investors during this timeframe, amidst notable price fluctuations and volatility.
- New investors looking at GOOG should consider market conditions, historical performance, and potential risks associated with stock price volatility.